# Настраевае среду colab

In [ ]:
!git clone https://github.com/meineac/counterfeit-ozon-ml.git

Cloning into 'counterfeit-ozon-ml'...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 62 (delta 23), reused 48 (delta 11), pack-reused 0 (from 0)
Receiving objects: 100% (62/62), 38.89 KiB | 7.78 MiB/s, done.
Resolving deltas: 100% (23/23), done.


In [ ]:
%cd counterfeit-ozon-ml

/content/counterfeit-ozon-ml


In [ ]:
!git pull

Already up to date.


In [ ]:
%pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.4/247.4 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.7/264.7 kB 15.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir -p data/raw
!cp "/content/drive/MyDrive/OZON_ML/ml_ozon_сounterfeit_data/ml_ozon_сounterfeit_test_new.csv" data/raw/ml_ozon_counterfeit_test_new.csv
!cp "/content/drive/MyDrive/OZON_ML/ml_ozon_сounterfeit_data/ml_ozon_сounterfeit_train.csv" data/raw/ml_ozon_counterfeit_train.csv


In [ ]:
!cp "/content/drive/MyDrive/OZON_ML/ml_ozon_сounterfeit_data/ml_ozon_сounterfeit_test.csv" data/raw/ml_ozon_counterfeit_test.csv

# Solutions

## Загрузка данных train

In [ ]:
import pandas as pd
import numpy as np

file_folder = 'data/raw/'
file_name = 'ml_ozon_counterfeit_train.csv'
file_path = file_folder + file_name

try:
    df_train = pd.read_csv(file_path)
    print("✅ Тренировочный файл успешно загружен.")
except FileNotFoundError:
    print(f"❌ Ошибка: Файл не найден по пути '{file_path}'. Проверьте путь.")

if 'df_train' in locals():
    print("\n--- 1. Размерность данных ---")
    print(f"Форма датасета: {df_train.shape}")

    print("\n--- 2. Общая информация и типы данных ---")
    # .info() показывает типы данных и количество НЕпустых значений
    df_train.info()

✅ Тренировочный файл успешно загружен.

--- 1. Размерность данных ---
Форма датасета: (197198, 45)

--- 2. Общая информация и типы данных ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197198 entries, 0 to 197197
Data columns (total 45 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   id                            197198 non-null  int64  
 1   resolution                    197198 non-null  int64  
 2   brand_name                    116667 non-null  object 
 3   description                   171138 non-null  object 
 4   name_rus                      197198 non-null  object 
 5   CommercialTypeName4           197198 non-null  object 
 6   rating_1_count                47193 non-null   float64
 7   rating_2_count                47193 non-null   float64
 8   rating_3_count                47193 non-null   float64
 9   rating_4_count                47193 non-null   float64
 10  rating_5_count        

## Предобработка данных

In [ ]:
import re # Импортируем модуль для работы с регулярными выражениями

def preprocess_data(df: pd.DataFrame) -> pd.DataFrame:
    print("--- Начинаю Шаг 3: Предобработка и инжиниринг признаков ---")

    # Создаем копию, чтобы не изменять оригинальный датафрейм
    df_processed = df.copy()

    # --- 1. Обработка пропущенных значений ---
    print("1. Заполняю пропуски...")

    # Блок с рейтингами и отзывами (NaN -> 0)
    rating_cols = [col for col in df_processed.columns if 'rating' in col or 'count' in col]
    # Уточняем список, чтобы не затронуть лишние столбцы
    cols_to_fill_zero = [
        'rating_1_count', 'rating_2_count', 'rating_3_count', 'rating_4_count', 'rating_5_count',
        'comments_published_count', 'photos_published_count', 'videos_published_count',
        'GmvTotal7', 'GmvTotal30', 'GmvTotal90', 'ExemplarAcceptedCountTotal7',
        'ExemplarAcceptedCountTotal30', 'ExemplarAcceptedCountTotal90', 'OrderAcceptedCountTotal7',
        'OrderAcceptedCountTotal30', 'OrderAcceptedCountTotal90', 'ExemplarReturnedCountTotal7',
        'ExemplarReturnedCountTotal30', 'ExemplarReturnedCountTotal90', 'ExemplarReturnedValueTotal7',
        'ExemplarReturnedValueTotal30', 'ExemplarReturnedValueTotal90', 'ItemVarietyCount', 'ItemAvailableCount'
    ]
    for col in cols_to_fill_zero:
        if col in df_processed.columns:
            df_processed[col] = df_processed[col].fillna(0)

    # Категориальные признаки
    df_processed['brand_name'] = df_processed['brand_name'].fillna('_UNKNOWN_')
    df_processed['description'] = df_processed['description'].fillna('no_description')

    print("   ...пропуски заполнены.")

    # --- 2. Очистка текста ---
    print("2. Очищаю текстовые поля от HTML-тегов...")
    def clean_html(text):
        if isinstance(text, str):
            # Удаляем HTML-теги
            clean_text = re.sub(r'<.*?>', ' ', text)
            # Удаляем переносы строк и лишние пробелы
            clean_text = re.sub(r'\s+', ' ', clean_text).strip()
            return clean_text
        return text

    df_processed['description_cleaned'] = df_processed['description'].apply(clean_html)
    df_processed['name_rus_cleaned'] = df_processed['name_rus'].apply(clean_html)
    print("   ...текст очищен.")

    # --- 3. Feature Engineering ---
    print("3. Создаю новые признаки...")

    # Признаки на основе длины текста
    df_processed['description_len'] = df_processed['description_cleaned'].str.len()
    df_processed['name_rus_len'] = df_processed['name_rus_cleaned'].str.len()

    # Признаки-отношения (добавляем epsilon для избежания деления на ноль)
    epsilon = 1e-6
    df_processed['return_to_sales_ratio_90'] = df_processed['item_count_returns90'] / (df_processed['item_count_sales90'] + epsilon)
    df_processed['fake_return_ratio_90'] = df_processed['item_count_fake_returns90'] / (df_processed['item_count_returns90'] + epsilon)

    # Признак "есть ли у товара отзывы"
    df_processed['has_reviews'] = (df_processed['rating_1_count'] + df_processed['rating_5_count'] > 0).astype(int)
    print("   ...новые признаки созданы.")

    print("\n--- Проверка результата ---")
    # Проверяем, что пропусков не осталось (кроме тех, где изначально не было)
    remaining_na = df_processed.isnull().sum()
    print("Оставшиеся пропуски:")
    print(remaining_na[remaining_na > 0])
    print("\n✅ Шаг 3 успешно завершен!")
    return df_processed

if 'df_train' in locals():
    df_processed = preprocess_data(df_train)
else:
    print("❌ Переменная 'df_train' не найдена.")

--- Начинаю Шаг 3: Предобработка и инжиниринг признаков ---
1. Заполняю пропуски...
   ...пропуски заполнены.
2. Очищаю текстовые поля от HTML-тегов...
   ...текст очищен.
3. Создаю новые признаки...
   ...новые признаки созданы.

--- Проверка результата ---
Оставшиеся пропуски:
Series([], dtype: int64)

✅ Шаг 3 успешно завершен!


### Сохраняем обработанные данные

In [ ]:
import os

if 'df_processed' in locals():
    # --- ШАГ 1: Определяем путь для сохранения ---
    # Создаем папку 'data/processed/', если она еще не существует
    processed_data_path = '/content/counterfeit-ozon-ml/data/processed/'
    os.makedirs(processed_data_path, exist_ok=True)

    # Имя файла
    output_file_path = os.path.join(processed_data_path, 'df_processed.feather')

    # --- ШАГ 2: Сохраняем DataFrame ---
    try:
        # Feather требует сбросить индекс, если он не является стандартным RangeIndex
        df_processed.reset_index(drop=True).to_feather(output_file_path)
        print(f"✅ Обработанные данные успешно сохранены в файл:")
        print(f"   -> {output_file_path}")
    except Exception as e:
        print(f"❌ Произошла ошибка при сохранении файла: {e}")

else:
    print("❌ Переменная 'df_processed' не найдена. Пожалуйста, выполните предыдущий шаг.")

✅ Обработанные данные успешно сохранены в файл:
   -> /content/counterfeit-ozon-ml/data/processed/df_processed.feather


## Расшириные признаки

## Обучение модели

### # --- 1. Подготовка данных ---

In [ ]:
# --- Применяем новую функцию к нашим данным ---
if 'df_processed' in locals():
    df_processed_advanced = create_advanced_features(df_processed)

    print("\n--- Перезапускаю обучение ансамбля с новыми признаками ---")

    df_processed_advanced['text_features'] = df_processed_advanced['name_rus_cleaned'] + ' ' + df_processed_advanced['description_cleaned']
    y = df_processed_advanced['resolution']

    features_to_drop = ['resolution', 'id', 'description', 'name_rus', 'description_cleaned', 'name_rus_cleaned', 'text_features']
    X_tab = df_processed_advanced.drop(columns=features_to_drop)
    categorical_features = ['brand_name', 'CommercialTypeName4']
    for col in categorical_features:
        X_tab[col] = X_tab[col].astype('category')

    X_text = df_processed_advanced['text_features']
else:
    print("❌ Переменная 'df_processed' не найдена.")

   ...создаю продвинутые признаки...
   ...продвинутые признаки созданы.

--- Перезапускаю обучение ансамбля с новыми признаками ---


### --- 2. Кросс-валидация ---

In [ ]:
required_vars = ['y', 'X_tab', 'X_text', 'categorical_features']
error = [var for var in required_vars if var not in locals()]

if not error:
    N_SPLITS = 5
    skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

    oof_lgbm = np.zeros(len(X_tab))
    oof_text = np.zeros(len(X_tab))

    print(f"\nПолучаю OOF-предсказания на {N_SPLITS} фолдах (LGBM будет использовать новые фичи)...")
    for fold, (train_idx, val_idx) in enumerate(skf.split(X_tab, y)):
        print(f"--- Фолд {fold+1}/{N_SPLITS} ---")

        # Обучение LightGBM
        X_train_tab, y_train = X_tab.iloc[train_idx], y.iloc[train_idx]
        X_val_tab, y_val = X_tab.iloc[val_idx], y.iloc[val_idx]
        neg_count, pos_count = y_train.value_counts()
        lgbm_model = LGBMClassifier(random_state=42, scale_pos_weight=neg_count / pos_count, n_estimators=1000, learning_rate=0.05, num_leaves=31)
        lgbm_model.fit(X_train_tab, y_train, eval_set=[(X_val_tab, y_val)], eval_metric='f1', callbacks=[], categorical_feature=categorical_features)
        oof_lgbm[val_idx] = lgbm_model.predict_proba(X_val_tab)[:, 1]

        # Обучение Text Pipeline
        X_train_text, X_val_text = X_text.iloc[train_idx], X_text.iloc[val_idx]
        text_pipeline = Pipeline([
            ('tfidf', TfidfVectorizer(max_features=20000, ngram_range=(1, 2), min_df=3, max_df=0.9)),
            ('logreg', LogisticRegression(C=5, class_weight='balanced', random_state=42, solver='liblinear'))
        ])
        text_pipeline.fit(X_train_text, y_train)
        oof_text[val_idx] = text_pipeline.predict_proba(X_val_text)[:, 1]
else:
    print(f"❌ Не найдены переменные: {', '.join(error)}")


Получаю OOF-предсказания на 5 фолдах (LGBM будет использовать новые фичи)...
--- Фолд 1/5 ---


KeyboardInterrupt: 

### --- 3. Поиск лучшего веса и порога ---

In [ ]:
required_vars = ['oof_lgbm', 'oof_text']
error = [var for var in required_vars if var not in locals()]

if not error:
    print("\nПодбираю лучший вес и порог для НОВОГО ансамбля...")
    best_f1, best_weight, best_threshold = 0, 0, 0
    for weight in np.arange(0.6, 1.01, 0.05):
        blended_oof = weight * oof_lgbm + (1 - weight) * oof_text
        for threshold in np.arange(0.1, 0.91, 0.05):
            preds = (blended_oof > threshold).astype(int)
            f1 = f1_score(y, preds)
            if f1 > best_f1:
                best_f1, best_weight, best_threshold = f1, weight, threshold

    print("\n" + "="*50)
    print("Подбор для УЛУЧШЕННОЙ модели завершен!")
    print(f"F1-score УЛУЧШЕННОЙ табличной модели (LGBM): {f1_score(y, (oof_lgbm > 0.5).astype(int)):.4f}")
    print(f"F1-score текстовой модели (LogReg): {f1_score(y, (oof_text > 0.5).astype(int)):.4f}")
    print("-" * 20)
    print(f"Лучший F1-score НОВОГО ансамбля: {best_f1:.4f}")
    print(f"Лучший вес для LGBM: {best_weight:.2f}")
    print(f"Лучший порог: {best_threshold:.2f}")
    print("="*50)
else:
    print(f"❌ Не найдены переменные: {', '.join(error)}")


Подбираю лучший вес и порог для НОВОГО ансамбля...

Подбор для УЛУЧШЕННОЙ модели завершен!
F1-score УЛУЧШЕННОЙ табличной модели (LGBM): 0.8007
F1-score текстовой модели (LogReg): 0.6260
--------------------
Лучший F1-score НОВОГО ансамбля: 0.8328
Лучший вес для LGBM: 0.70
Лучший порог: 0.70


### Тонкая настройка гиперпараметров с помощью Optuna

In [ ]:
import optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
import numpy as np
import joblib  # Для сохранения/загрузки study
import json    # Для сохранения параметров
import os

if 'df_processed_advanced' in locals() and 'X_tab' in locals() and 'y' in locals():
    print("--- Начинаю Этап 1: Поиск лучших гиперпараметров для LightGBM с Optuna ---")

    def objective(trial):
        params = {
            'objective': 'binary', 'metric': 'f1', 'verbosity': -1,
            'boosting_type': 'gbdt', 'random_state': 42, 'n_estimators': 1000,
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
            'num_leaves': trial.suggest_int('num_leaves', 20, 300),
            'max_depth': trial.suggest_int('max_depth', 3, 12),
            'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
            'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        }
        neg_count, pos_count = y.value_counts()
        params['scale_pos_weight'] = neg_count / pos_count

        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        f1_scores = []
        for train_idx, val_idx in skf.split(X_tab, y):
            X_train, y_train = X_tab.iloc[train_idx], y.iloc[train_idx]
            X_val, y_val = X_tab.iloc[val_idx], y.iloc[val_idx]
            model = LGBMClassifier(**params)
            model.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='f1', callbacks=[])
            preds = model.predict(X_val)
            f1_scores.append(f1_score(y_val, preds))
        return np.mean(f1_scores)

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=50, show_progress_bar=True)

    print("\n" + "="*50)
    print("Поиск завершен!")
    print(f"Лучший F1-score: {study.best_value:.4f}")
    print("Лучшие гиперпараметры:")
    for key, value in study.best_params.items():
        print(f"  {key}: {value}")
    print("="*50)

    # --- 2. Сохранение результатов ---
    print("\n--- Сохраняю результаты исследования ---")

    # Путь для сохранения на Google Drive
    save_path = '/content/drive/MyDrive/Colab Notebooks/OzonContest'
    os.makedirs(save_path, exist_ok=True)

    # Сохраняем весь объект study
    study_file = os.path.join(save_path, 'optuna_study.pkl')
    joblib.dump(study, study_file)
    print(f"✅ Полное исследование сохранено в: {study_file}")

    # Сохраняем лучшие параметры в json
    params_file = os.path.join(save_path, 'best_lgbm_params.json')
    with open(params_file, 'w') as f:
        json.dump(study.best_params, f, indent=4)
    print(f"✅ Лучшие параметры сохранены в: {params_file}")

else:
    print("❌ Необходимые данные (df_processed_advanced, X_tab, y) не найдены.")

ModuleNotFoundError: No module named 'optuna'

### Применение оптимизированных параметров и оценка

In [17]:
import json
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from google.colab import drive

# --- 1. Загрузка лучших параметров ---
print("--- Загружаю лучшие параметры, найденные Optuna ---")
try:
    drive.mount('/content/drive', force_remount=True)
    params_file = '/content/drive/MyDrive/Colab Notebooks/OzonContest/best_lgbm_params.json'
    with open(params_file, 'r') as f:
        best_lgbm_params = json.load(f)
    print("✅ Лучшие параметры успешно загружены:")
    print(best_lgbm_params)
except Exception as e:
    print(f"❌ Ошибка загрузки файла с параметрами: {e}")

# --- 2. Перезапуск ансамбля с новыми параметрами ---
if 'df_processed_advanced' in locals() and 'best_lgbm_params' in locals():
    print("\n--- Перезапускаю обучение ансамбля с ОПТИМИЗИРОВАННЫМИ параметрами ---")

    # Подготовка данных
    y = df_processed_advanced['resolution']
    X_tab = df_processed_advanced.drop(columns=['resolution', 'id', 'description', 'name_rus', 'description_cleaned', 'name_rus_cleaned', 'text_features'])
    X_text = df_processed_advanced['text_features']
    categorical_features = ['brand_name', 'CommercialTypeName4']
    for col in categorical_features:
        X_tab[col] = X_tab[col].astype('category')

    N_SPLITS = 5
    skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
    oof_lgbm = np.zeros(len(X_tab))
    oof_text = np.zeros(len(X_tab))

    for fold, (train_idx, val_idx) in enumerate(skf.split(X_tab, y)):
        print(f"--- Фолд {fold+1}/{N_SPLITS} ---")
        X_train_tab, y_train = X_tab.iloc[train_idx], y.iloc[train_idx]
        X_val_tab, y_val = X_tab.iloc[val_idx], y.iloc[val_idx]

        # Обучение ОПТИМИЗИРОВАННОГО LightGBM
        final_params = best_lgbm_params.copy()
        neg_count, pos_count = y_train.value_counts()
        final_params.update({
            'objective': 'binary', 'metric': 'f1', 'verbosity': -1,
            'random_state': 42, 'n_estimators': 2000,
            'scale_pos_weight': neg_count / pos_count
        })
        lgbm_model = LGBMClassifier(**final_params)
        lgbm_model.fit(X_train_tab, y_train, eval_set=[(X_val_tab, y_val)], eval_metric='f1', callbacks=[]) # early_stopping(100)
        oof_lgbm[val_idx] = lgbm_model.predict_proba(X_val_tab)[:, 1]

        # Обучение текстовой модели
        X_train_text, X_val_text = X_text.iloc[train_idx], X_text.iloc[val_idx]

        # --- ИСПРАВЛЕНИЕ: Определяем пайплайн ВНУТРИ цикла ---
        text_pipeline = Pipeline([
            ('tfidf', TfidfVectorizer(max_features=20000, ngram_range=(1, 2), min_df=3, max_df=0.9)),
            ('logreg', LogisticRegression(C=5, class_weight='balanced', random_state=42, solver='liblinear'))
        ])
        # ----------------------------------------------------

        text_pipeline.fit(X_train_text, y_train)
        oof_text[val_idx] = text_pipeline.predict_proba(X_val_text)[:, 1]

    # --- 3. Поиск лучшего веса и порога ---
    print("\nПодбираю лучший вес и порог для ФИНАЛЬНОГО ансамбля...")
    best_f1, best_weight, best_threshold = 0, 0, 0
    for weight in np.arange(0.6, 1.01, 0.05):
        blended_oof = weight * oof_lgbm + (1 - weight) * oof_text
        for threshold in np.arange(0.1, 0.91, 0.05):
            preds = (blended_oof > threshold).astype(int)
            f1 = f1_score(y, preds)
            if f1 > best_f1:
                best_f1, best_weight, best_threshold = f1, weight, threshold

    print("\n" + "="*50)
    print("Подбор для ОПТИМИЗИРОВАННОЙ модели завершен!")
    print(f"Наш прошлый лучший результат: 0.8328")
    print("-" * 20)
    print(f"НОВЫЙ лучший F1-score ансамбля: {best_f1:.4f}")
    print(f"Лучший вес для LGBM: {best_weight:.2f}")
    print(f"Лучший порог: {best_threshold:.2f}")
    print("="*50)
else:
    print("❌ Необходимые данные (df_processed_advanced) не найдены.")

--- Загружаю лучшие параметры, найденные Optuna ---
Mounted at /content/drive
✅ Лучшие параметры успешно загружены:
{'learning_rate': 0.05905048835314297, 'num_leaves': 263, 'max_depth': 12, 'min_child_samples': 7, 'subsample': 0.8288864010163431, 'colsample_bytree': 0.6048620922810927, 'reg_alpha': 0.156919415649457, 'reg_lambda': 0.8711271060286511}

--- Перезапускаю обучение ансамбля с ОПТИМИЗИРОВАННЫМИ параметрами ---
--- Фолд 1/5 ---
--- Фолд 2/5 ---
--- Фолд 3/5 ---
--- Фолд 4/5 ---
--- Фолд 5/5 ---

Подбираю лучший вес и порог для ФИНАЛЬНОГО ансамбля...

Подбор для ОПТИМИЗИРОВАННОЙ модели завершен!
Наш прошлый лучший результат: 0.8328
--------------------
НОВЫЙ лучший F1-score ансамбля: 0.8477
Лучший вес для LGBM: 0.60
Лучший порог: 0.55


## Обучение и формирование submission.csv

### --- 1. Загрузка и полная обработка данных ---

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from rapidfuzz import fuzz

print("--- Создаю финальный submission-файл с лучшей моделью ---")
print("1. Загружаю и обрабатываю данные...")


df_train = pd.read_csv('data/raw/ml_ozon_сounterfeit_train.csv')
df_test = pd.read_csv('data/raw/ml_ozon_сounterfeit_test.csv')

test_ids = df_test['id']

train_processed = preprocess_data(df_train)
train_final = create_advanced_features(train_processed)

test_processed = preprocess_data(df_test)
test_final = create_advanced_features(test_processed)

### --- 2. Обучение финальных моделей на ВСЕХ данных ---

In [ ]:
required_vars = ['train_final', 'test_final']
error = [var for var in required_vars if var not in locals()]

if not error:
    print("2. Обучаю финальные модели...")

    # LightGBM
    train_final['text_features'] = train_final['name_rus_cleaned'] + ' ' + train_final['description_cleaned']
    features_to_drop = ['resolution', 'id', 'description', 'name_rus', 'description_cleaned', 'name_rus_cleaned', 'text_features']
    X_train_tab = train_final.drop(columns=features_to_drop)
    y_train = train_final['resolution']

    test_final['text_features'] = test_final['name_rus_cleaned'] + ' ' + test_final['description_cleaned']
    X_test_tab = test_final.drop(columns=[col for col in features_to_drop if col in test_final.columns])

    X_train_tab, X_test_tab = X_train_tab.align(X_test_tab, join='left', axis=1, fill_value=0)
    categorical_features = ['brand_name', 'CommercialTypeName4']
    for col in categorical_features:
        X_train_tab[col] = X_train_tab[col].astype('category')
        X_test_tab[col] = X_test_tab[col].astype('category')

    neg_count, pos_count = y_train.value_counts()
    lgbm_model = LGBMClassifier(random_state=42, scale_pos_weight=neg_count / pos_count, n_estimators=1000, learning_rate=0.05, num_leaves=31)
    lgbm_model.fit(X_train_tab, y_train, categorical_feature=categorical_features)
    print("   ...LGBM обучен.")

    # Текстовая модель
    X_train_text = train_final['text_features']
    X_test_text = test_final['text_features']

    text_pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(max_features=20000, ngram_range=(1, 2), min_df=3, max_df=0.9)),
        ('logreg', LogisticRegression(C=5, class_weight='balanced', random_state=42, solver='liblinear'))
    ])
    text_pipeline.fit(X_train_text, y_train)
    print("   ...Текстовая модель обучена.")
else:
    print(f"❌ Не найдены переменные: {', '.join(error)}")

### --- 3. Предсказание и блендинг ---

In [ ]:
required_vars = ['lgbm_model', 'text_pipeline']
error = [var for var in required_vars if var not in locals()]

if not error:
    print("3. Делаю предсказания и смешиваю их...")
    lgbm_probs = lgbm_model.predict_proba(X_test_tab)[:, 1]
    text_probs = text_pipeline.predict_proba(X_test_text)[:, 1]

    LGBM_WEIGHT = 0.70
    TEXT_WEIGHT = 1 - LGBM_WEIGHT
    blended_probs = LGBM_WEIGHT * lgbm_probs + TEXT_WEIGHT * text_probs
else:
    print(f"❌ Не найдены переменные: {', '.join(error)}")

### --- 4. Применение порога и сохранение ---

In [ ]:
from datetime import datetime

required_vars = ['blended_probs']
error = [var for var in required_vars if var not in locals()]

if not error:
    print("4. Применяю порог и сохраняю результат...")
    BEST_THRESHOLD = 0.70
    predictions = (blended_probs > BEST_THRESHOLD).astype(int)

    submission = pd.DataFrame({'id': test_ids, 'prediction': predictions})

    # Создаём имя файла с меткой времени
    timestamp = datetime.now().strftime("%m.%d_%H:%M")
    filename = f"submission_{timestamp}.csv"
    # Сохраняем
    submission.to_csv(filename, index=False)

    print("✅ Финальный сабмит 'submission.csv' готов к отправке!")
else:
    print(f"❌ Не найдены переменные: {', '.join(error)}")